In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math

In [2]:
# def init_graph(G,node_adj_frame):
#     G.add_nodes_from([i for i in range(len(node_adj_frame))])
#     labels = {}
#     labels = node_adj_frame.columns
#     for i in range(len(node_adj_frame)):
#         snode = -1
#         if node_adj_frame[labels[0]][i] < 81:
#             snode= node_adj_frame[labels[0]][i]-1 #s.no.
#         else:
#             snode= node_adj_frame[labels[0]][i]-2
#         temp = node_adj_frame[labels[2]][i]
#         if ',' in str(temp):
#             sedge_arr = temp.split(',')
#             for j in range(0, len(sedge_arr)):
#                 k = int(sedge_arr[j])
#                 if k < 81:
#                     G.add_edge(snode, k-1)
#                 else:
#                     G.add_edge(snode, k-2)
#         elif temp == np.nan:
#             print("ERROR: Not found in the adjacency excel sheet")
#         else:
#             G.add_edge(snode, int(temp)-1)
#     return

def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        if snode == 80:
            continue
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    G.remove_node(80)
    return

In [3]:
# def init_graph_attr(G,AdjFile,df,col1,col2, col3):
#     node_adj_frame = pd.read_excel(AdjFile)
#     node_list = node_adj_frame["KGISTalukN"].tolist()
#     nodeAttr = {}
#     init_graph(G,node_adj_frame)
#     capability_vector = list(zip(df[col1],df[col2],df[col3]))
#     node_attri_dict = dict(zip(df["Taluka"],capability_vector))
#     node_attri_dict = dict((k.lower(), v) for k, v in node_attri_dict.items())
#     for i in range(len(node_adj_frame)):
#         temp = {}
#         temp["capabilityvector"] = node_attri_dict[node_list[i].lower()]
#         temp["nodeStress"] = 0
#         temp["name"] = node_list[i]
#         nodeAttr[i] = temp
#     nx.set_node_attributes(G, nodeAttr)

def init_graph_attr(G,AdjFile,df,col1, col2, col3):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["KGISTalukN"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G,node_adj_frame)
    capability_vector = list(zip(df[col1], df[col2], df[col3]))
    node_attri_dict = dict(zip(df["Taluka"],capability_vector))
    node_attri_dict = dict((k.lower(), v) for k, v in node_attri_dict.items())
    for i in range(len(node_adj_frame)):
        temp = {}
        if i == 80:
            continue
        temp["capabilityvector"] = node_attri_dict[node_list[i].lower()]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp
    nt = {}
    nt["capabilityvector"] = node_attri_dict[node_list[226].lower()]
    nt["nodeStress"] = 0
    nt["name"] = "Hadagali"
    nodeAttr[226] = nt
    nx.set_node_attributes(G, nodeAttr)

In [4]:
G = nx.Graph()
df = pd.read_excel('Combined3D.xlsx')

In [5]:
dist_taluka_dict = defaultdict(list)
for k, v in zip(df["District_GIS"], df["Taluka"]):
    dist_taluka_dict[k].append(v)

In [6]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1
def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

In [7]:
def get_node_stress(G):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0,0,0]
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            # divide by 3?
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"].lower()]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [8]:
df['Taluka'] = df['Taluka'].apply(str.lower)
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df, "Normalized MMR", "Normalized IMR", "Normalized PAW")
initialstress = get_node_stress(G)

In [9]:
df["Initial Stress"] = df["Taluka"].map(initialstress)

In [10]:
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses-20%)", "Normalized MMR (HM -20%)", "Normalized PAW (HM-20%)")
KHminus20stress = get_node_stress(G)
df["Stress(KH-20%)"] = df["Taluka"].map(KHminus20stress)

In [11]:
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses-10%)", "Normalized MMR (HM -10%)", "Normalized PAW (HM-10%)")
KHminus10stress = get_node_stress(G)
df["Stress(KH-10%)"] = df["Taluka"].map(KHminus10stress)

In [12]:
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses+10%)", "Normalized MMR (HM +10%)", "Normalized PAW (HM+10%)")
KHplus10stress = get_node_stress(G)
df["Stress(KH+10%)"] = df["Taluka"].map(KHplus10stress)

In [13]:
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses+20%)", "Normalized MMR (HM +20%)", "Normalized PAW (HM+20%)")
KHplus20stress = get_node_stress(G)
df["Stress(KH+20%)"] = df["Taluka"].map(KHplus20stress)

In [14]:
df.to_excel("3D_StressFile.xlsx")

In [14]:
def aggregate(taluka_stress_dict):
    dist_stress = {}
    for dist, taluks in dist_taluka_dict.items():
        agg_stress = 0
        for taluk in taluks:
            try:
                agg_stress = agg_stress + taluka_stress_dict[taluk.lower()]
            except(KeyError):
                pass
        dist_stress[dist] = agg_stress/len(taluks)
    return dist_stress

In [15]:
df_imp = pd.read_excel('3d_Impact_scaled.xlsx')

In [16]:
combined_Impact_HM = pd.DataFrame()
aggregate_df = pd.DataFrame()

In [21]:
# ===================================================FOR HM -20%==============================================================
# Getting the graph ready for HM -20%
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses-20%)", "Normalized MMR (HM -20%)", "Normalized PAW (HM-20%)")

# These dict have the names vs Impact Score for HM -20%
IMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["IMR HI -20% | Impact"]))
MMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["MMR HI -20% | Impact"]))
PAW_impact_dict = dict(zip(df_imp["Taluka"],df_imp["PAW HI -20% | Impact"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
# IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
# MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())
# PAW_impact_dict= dict((k.lower(), v) for k, v in PAW_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)
aggregate_PAW_Impact = aggregate(PAW_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_PAW_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (HM - 20%)', 1:'IMPACT_SCORE_MMR (HM - 20%)', 2:'IMPACT_SCORE_PAW (HM - 20%)', 3:'STRESS_SCORE (HM -20%)'}, inplace = True)
aggregate_df = aI_df

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])
combined_PAW = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdPAW = combined_PAW.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()
trdPAW = combined_PAW.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (HM - 20%)', 1:'STRESS_SCORE (HM -20%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (HM - 20%)', 1:'STRESS_SCORE (HM -20%)'}, inplace = True)
trdPAW.rename(columns = {0:'IMPACT_SCORE_PAW (HM - 20%)', 1:'STRESS_SCORE (HM -20%)'}, inplace = True)

# Filling the dataframe
combined_Impact_HM = trdIMR
combined_Impact_HM['IMPACT_SCORE_MMR (HM - 20%)'] = trdMMR['IMPACT_SCORE_MMR (HM - 20%)']
combined_Impact_HM['IMPACT_SCORE_PAW (HM - 20%)'] = trdPAW['IMPACT_SCORE_PAW (HM - 20%)']

In [22]:
aggregate_df

,IMPACT_SCORE_IMR (HM - 20%),IMPACT_SCORE_MMR (HM - 20%),IMPACT_SCORE_PAW (HM - 20%),STRESS_SCORE (HM -20%)
Kalburgi,0.000000,0.000000,0.000000,0.285752
Chikkamagaluru,0.000000,0.000000,0.000000,0.325016
Dharwad,0.000000,0.000000,0.000000,0.397946
Hassan,0.000000,0.000000,0.000000,0.397411
Bengaluru_Urban,0.000000,0.000000,0.000000,0.209030
Uttara_Kannada,0.000000,0.000000,0.000000,0.275251
Belagavi,0.000000,0.000000,0.000000,0.218534
Bidar,0.000000,0.000000,0.000000,0.231000
Vijayapura,0.000000,0.000000,0.000000,0.186678
Bagalkot,0.000000,0.000000,0.000000,0.287923


In [41]:
# ===================================================FOR HM -10%==============================================================
# Getting the graph ready for HM -10%
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses-10%)", "Normalized MMR (HM -10%)", "Normalized PAW (HM-10%)")

# These dict have the names vs Impact Score for HM -10%
IMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["IMR HI -10% | Impact"]))
MMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["MMR HI -10% | Impact"]))
PAW_impact_dict = dict(zip(df_imp["Taluka"],df_imp["PAW HI -10% | Impact"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())
PAW_impact_dict= dict((k.lower(), v) for k, v in PAW_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)
aggregate_PAW_Impact = aggregate(PAW_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_PAW_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (HM - 10%)', 1:'IMPACT_SCORE_MMR (HM - 10%)', 2:'IMPACT_SCORE_PAW (HM - 10%)', 3:'STRESS_SCORE (HM -10%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (HM - 10%)'] = aI_df['IMPACT_SCORE_IMR (HM - 10%)']
aggregate_df['IMPACT_SCORE_MMR (HM - 10%)'] = aI_df['IMPACT_SCORE_MMR (HM - 10%)']
aggregate_df['IMPACT_SCORE_PAW (HM - 10%)'] = aI_df['IMPACT_SCORE_PAW (HM - 10%)']
aggregate_df['STRESS_SCORE (HM -10%)'] = aI_df['STRESS_SCORE (HM -10%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])
combined_PAW = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdPAW = combined_PAW.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()
trdPAW = combined_PAW.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (HM - 10%)', 1:'STRESS_SCORE (HM -10%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (HM - 10%)', 1:'STRESS_SCORE (HM -10%)'}, inplace = True)
trdPAW.rename(columns = {0:'IMPACT_SCORE_PAW (HM - 10%)', 1:'STRESS_SCORE (HM -10%)'}, inplace = True)

# Filling the dataframe
combined_Impact_HM['STRESS_SCORE (HM -10%)'] = trdPAW['STRESS_SCORE (HM -10%)']
combined_Impact_HM['IMPACT_SCORE_IMR (HM - 10%)'] = trdIMR['IMPACT_SCORE_IMR (HM - 10%)']
combined_Impact_HM['IMPACT_SCORE_MMR (HM - 10%)'] = trdMMR['IMPACT_SCORE_MMR (HM - 10%)']
combined_Impact_HM['IMPACT_SCORE_PAW (HM - 10%)'] = trdPAW['IMPACT_SCORE_PAW (HM - 10%)']

In [42]:
# ===================================================FOR HM +10%==============================================================
# Getting the graph ready for HM +10%
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses+10%)", "Normalized MMR (HM +10%)", "Normalized PAW (HM+10%)")

# These dict have the names vs Impact Score for HM +10%
IMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["IMR HI +10% | Impact"]))
MMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["MMR HI +10% | Impact"]))
PAW_impact_dict = dict(zip(df_imp["Taluka"],df_imp["PAW HI +10% | Impact"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())
PAW_impact_dict= dict((k.lower(), v) for k, v in PAW_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)
aggregate_PAW_Impact = aggregate(PAW_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_PAW_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (HM + 10%)', 1:'IMPACT_SCORE_MMR (HM + 10%)', 2:'IMPACT_SCORE_PAW (HM + 10%)', 3:'STRESS_SCORE (HM +10%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (HM + 10%)'] = aI_df['IMPACT_SCORE_IMR (HM + 10%)']
aggregate_df['IMPACT_SCORE_MMR (HM + 10%)'] = aI_df['IMPACT_SCORE_MMR (HM + 10%)']
aggregate_df['IMPACT_SCORE_PAW (HM + 10%)'] = aI_df['IMPACT_SCORE_PAW (HM + 10%)']
aggregate_df['STRESS_SCORE (HM +10%)'] = aI_df['STRESS_SCORE (HM +10%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])
combined_PAW = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdPAW = combined_PAW.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()
trdPAW = combined_PAW.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (HM + 10%)', 1:'STRESS_SCORE (HM +10%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (HM + 10%)', 1:'STRESS_SCORE (HM +10%)'}, inplace = True)
trdPAW.rename(columns = {0:'IMPACT_SCORE_PAW (HM + 10%)', 1:'STRESS_SCORE (HM +10%)'}, inplace = True)

# Filling the dataframe
combined_Impact_HM['STRESS_SCORE (HM +10%)'] = trdPAW['STRESS_SCORE (HM +10%)']
combined_Impact_HM['IMPACT_SCORE_IMR (HM + 10%)'] = trdIMR['IMPACT_SCORE_IMR (HM + 10%)']
combined_Impact_HM['IMPACT_SCORE_MMR (HM + 10%)'] = trdMMR['IMPACT_SCORE_MMR (HM + 10%)']
combined_Impact_HM['IMPACT_SCORE_PAW (HM + 10%)'] = trdPAW['IMPACT_SCORE_PAW (HM + 10%)']

In [43]:
# ===================================================FOR HM +20%==============================================================
# Getting the graph ready for HM +20%
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses+20%)", "Normalized MMR (HM +20%)", "Normalized PAW (HM+20%)")

# These dict have the names vs Impact Score for HM +20%
IMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["IMR HI +20% | Impact"]))
MMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["MMR HI +20% | Impact"]))
PAW_impact_dict = dict(zip(df_imp["Taluka"],df_imp["PAW HI +20% | Impact"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())
PAW_impact_dict= dict((k.lower(), v) for k, v in PAW_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)
aggregate_PAW_Impact = aggregate(PAW_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_PAW_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (HM + 20%)', 1:'IMPACT_SCORE_MMR (HM + 20%)', 2:'IMPACT_SCORE_PAW (HM + 20%)', 3:'STRESS_SCORE (HM +20%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (HM + 20%)'] = aI_df['IMPACT_SCORE_IMR (HM + 20%)']
aggregate_df['IMPACT_SCORE_MMR (HM + 20%)'] = aI_df['IMPACT_SCORE_MMR (HM + 20%)']
aggregate_df['IMPACT_SCORE_PAW (HM + 20%)'] = aI_df['IMPACT_SCORE_PAW (HM + 20%)']
aggregate_df['STRESS_SCORE (HM +20%)'] = aI_df['STRESS_SCORE (HM +20%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])
combined_PAW = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdPAW = combined_PAW.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()
trdPAW = combined_PAW.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (HM + 20%)', 1:'STRESS_SCORE (HM +20%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (HM + 20%)', 1:'STRESS_SCORE (HM +20%)'}, inplace = True)
trdPAW.rename(columns = {0:'IMPACT_SCORE_PAW (HM + 20%)', 1:'STRESS_SCORE (HM +20%)'}, inplace = True)

# Filling the dataframe
combined_Impact_HM['STRESS_SCORE (HM +20%)'] = trdPAW['STRESS_SCORE (HM +20%)']
combined_Impact_HM['IMPACT_SCORE_IMR (HM + 20%)'] = trdIMR['IMPACT_SCORE_IMR (HM + 20%)']
combined_Impact_HM['IMPACT_SCORE_MMR (HM + 20%)'] = trdMMR['IMPACT_SCORE_MMR (HM + 20%)']
combined_Impact_HM['IMPACT_SCORE_PAW (HM + 20%)'] = trdPAW['IMPACT_SCORE_PAW (HM + 20%)']

In [45]:
aggregate_df.to_excel('Scaled_3D_tableau.xlsx')